------------------------------------------------------------

CREACIÓN DEL DATASET DE INCENDIOS: ----------------------

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm import tqdm

In [2]:
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """
""" NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO """

' NO EJECUTAR SI NO QUIERES PASAR 8HORAS ESPERANDO PARA TERMINAR TODO EL PROCESO '

In [3]:
# Carga y concatenación de archivos de datos anuales
archivos = [f"Data/data_original_fuegos_spain_2000_2023/modis_{year}_Spain.csv" for year in range(2008, 2024)]
dataframes = []
for archivo in tqdm(archivos, desc="Cargando datos"):
    df = pd.read_csv(archivo)
    dataframes.append(df)
    print(f"Archivo {archivo} cargado: {df.shape[0]} filas.")
df = pd.concat(dataframes, ignore_index=True)
print(f"DataFrame concatenado creado: {df.shape[0]} filas.")


Cargando datos: 100%|██████████| 16/16 [00:00<00:00, 105.72it/s]

Archivo Data/data_original_fuegos_spain_2000_2023/modis_2008_Spain.csv cargado: 3003 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2009_Spain.csv cargado: 4623 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2010_Spain.csv cargado: 2708 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2011_Spain.csv cargado: 5453 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2012_Spain.csv cargado: 6185 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2013_Spain.csv cargado: 3453 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2014_Spain.csv cargado: 2603 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2015_Spain.csv cargado: 3174 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2016_Spain.csv cargado: 3211 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2017_Spain.csv cargado: 5183 filas.
Archivo Data/data_original_fuegos_spain_2000_2023/modis_2018_Spain.csv cargado: 

In [4]:
# Guardar el DataFrame concatenado
df.to_csv('Data/Data_Fires.csv', index=False)
print("DataFrame concatenado guardado en 'Data/Data_Fires.csv'.")


DataFrame concatenado guardado en 'Data/Data_Fires.csv'.


In [5]:
from geopy.geocoders import Nominatim
from tqdm import tqdm
import pandas as pd
import json

# Función para cargar el DataFrame
def cargar_dataframe(path):
    try:
        df = pd.read_csv(path)
        filas_procesadas = df['location_info'].notna().sum()
        print(f"Reanudando desde la fila {filas_procesadas}.")
        return df, filas_procesadas
    except FileNotFoundError:
        print("Archivo no encontrado. Empezando desde el principio.")
        return pd.DataFrame(), 0

# Función para cargar o inicializar el cache
def cargar_cache(path):
    try:
        with open(path, 'r') as file:
            cache = json.load(file)
        print("Cache de geolocalización cargado.")
    except (FileNotFoundError, json.JSONDecodeError):
        cache = {}
    return cache

geolocator = Nominatim(user_agent="mi_aplicacion_geopy")
archivo_guardado = 'Data/Data_Fires_location.csv'
archivo_cache = 'Data/cache_geoloc.json'

# Cargar o inicializar DataFrame y cache
df, inicio = cargar_dataframe(archivo_guardado)
cache = cargar_cache(archivo_cache)

if df.empty:
    df_original = pd.read_csv('Data/Data_Fires.csv')
    df = df_original.copy()
    df['location_info'] = None

# Función modificada para obtener la localización
def get_location(row):
    if pd.notna(row['location_info']):
        return row['location_info']
    # Convertir las coordenadas a string para usarlas como clave en el diccionario
    coords_key = f"{row['latitude']},{row['longitude']}"
    if coords_key in cache:
        return cache[coords_key]
    else:
        try:
            location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
            result = location.address if location else None
            # Guardar el resultado usando la clave de coordenadas convertida a string
            cache[coords_key] = result
            return result
        except Exception as e:
            print(f"Error al obtener la geolocalización: {e}")
            return None

# Al guardar el cache, ya no deberías tener el problema del TypeError
for i in tqdm(range(inicio, len(df)), desc="Geolocalización inversa"):
    df.at[i, 'location_info'] = get_location(df.iloc[i])
    if i % 1000 == 0 or i == len(df) - 1:
        df.to_csv(archivo_guardado, index=False)
        with open(archivo_cache, 'w') as file:
            json.dump(cache, file)
        print(f"Datos y cache guardados hasta la fila {i}.")

print("Proceso completado y datos guardados.")

Archivo no encontrado. Empezando desde el principio.
Cache de geolocalización cargado.


Geolocalización inversa:   0%|          | 0/60536 [00:00<?, ?it/s]

Error al obtener la geolocalización: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=43.5265&lon=-5.734&format=json&addressdetails=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x126949010>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))


Geolocalización inversa:   0%|          | 1/60536 [00:03<61:34:31,  3.66s/it]

Datos y cache guardados hasta la fila 0.


...

Datos y cache guardados hasta la fila 59000.


Geolocalización inversa:  99%|█████████▉| 60001/60536 [8:18:32<06:21,  1.40it/s]

Datos y cache guardados hasta la fila 60000.


Geolocalización inversa: 100%|██████████| 60536/60536 [8:23:01<00:00,  2.01it/s]

Datos y cache guardados hasta la fila 60535.
Proceso completado y datos guardados.


In [25]:
# Realiza una segunda búsqueda para intentar completar las filas que no tienen información en la columna "location_info"

from geopy.geocoders import Nominatim
from tqdm import tqdm
import pandas as pd
import json

# Cargar el DataFrame
def cargar_dataframe(path):
    try:
        df = pd.read_csv(path)
        print(f"Total de filas en el DataFrame: {len(df)}.")
        filas_sin_procesar = df['location_info'].isna().sum()
        print(f"Filas sin información de localización: {filas_sin_procesar}.")
        return df, filas_sin_procesar
    except FileNotFoundError:
        print("Archivo no encontrado. Por favor, verifica la ruta del archivo.")
        return pd.DataFrame(), 0

# Cargar o inicializar el cache
def cargar_cache(path):
    try:
        with open(path, 'r') as file:
            cache = json.load(file)
        print("Cache de geolocalización cargado.")
    except (FileNotFoundError, json.JSONDecodeError):
        print("Cache no encontrado o corrupto. Se creará uno nuevo.")
        cache = {}
    return cache

geolocator = Nominatim(user_agent="mi_aplicacion_geopy")
archivo_guardado = 'Data/Data_Fires_location.csv'  # Asegúrate de ajustar esta ruta
archivo_cache = 'Data/cache_geoloc.json'  # Asegúrate de ajustar esta ruta

# Cargar o inicializar DataFrame y cache
df, filas_sin_procesar = cargar_dataframe(archivo_guardado)
cache = cargar_cache(archivo_cache)

# Si todas las filas ya tienen información, no hay necesidad de continuar
if filas_sin_procesar == 0:
    print("Todas las filas ya tienen información de localización.")
else:
    # Función para obtener la localización
    def get_location(row):
        coords_key = f"{row['latitude']},{row['longitude']}"
        if coords_key in cache:
            return cache[coords_key]
        else:
            try:
                location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
                result = location.address if location else None
                cache[coords_key] = result
                return result
            except Exception as e:
                print(f"Error al obtener la geolocalización: {e}")
                return None

    # Procesar solo las filas sin información de localización
    filas_a_procesar = df[df['location_info'].isna()]
    for i, row in tqdm(filas_a_procesar.iterrows(), total=filas_a_procesar.shape[0], desc="Geolocalización inversa"):
        df.at[i, 'location_info'] = get_location(row)

    # Guardar los cambios realizados en el DataFrame y el cache
    df.to_csv(archivo_guardado, index=False)
    with open(archivo_cache, 'w') as file:
        json.dump(cache, file)
    print("Proceso completado y datos guardados.")


/var/folders/7s/xkdgs46x3k5636bxx89xwtqh0000gn/T/ipykernel_4872/2834957895.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Total de filas en el DataFrame: 60536.
Filas sin información de localización: 22.
Cache de geolocalización cargado.


Geolocalización inversa: 100%|██████████| 22/22 [00:10<00:00,  2.04it/s]


Proceso completado y datos guardados.


In [6]:
import pandas as pd
from tqdm import tqdm

# Cargar el DataFrame
df = pd.read_csv('Data/Data_Fires_location.csv')

# Listas de provincias y comunidades autónomas de España
provincias = [
    "A Coruña", "Álava", "Albacete", "Alicante", "Almería", "Asturias", "Ávila", "Badajoz",
    "Barcelona", "Burgos", "Cáceres", "Cádiz", "Cantabria", "Castellón", "Ciudad Real",
    "Córdoba", "Cuenca", "Girona", "Granada", "Guadalajara", "Gipuzkoa", "Huelva", "Huesca",
    "Islas Baleares", "Jaén", "La Rioja", "Las Palmas", "León", "Lleida", "Lugo", "Madrid",
    "Málaga", "Murcia", "Navarra", "Ourense", "Palencia", "Pontevedra", "Salamanca", "Segovia",
    "Sevilla", "Soria", "Tarragona", "Santa Cruz de Tenerife", "Teruel", "Toledo", "Valencia",
    "Valladolid", "Vizcaya", "Zamora", "Zaragoza"
]

comunidades_autonomas = [
    "Andalucía", "Aragón", "Principado de Asturias", "Islas Baleares", "Canarias",
    "Cantabria", "Castilla y León", "Castilla-La Mancha", "Cataluña", "Extremadura",
    "Galicia", "Comunidad de Madrid", "Región de Murcia", "Comunidad Foral de Navarra",
    "País Vasco", "La Rioja", "Comunidad Valenciana", "Ceuta", "Melilla"
]

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Procesando ubicaciones"):
    # Convertir la dirección a cadena para asegurar la operación 'in'
    address = str(row['location_info'])
    
    # Buscar la provincia y comunidad autónoma solo si la dirección no es 'nan'
    if address != 'nan':
        provincia = next((p for p in provincias if p in address), None)
        comunidad_autonoma = next((c for c in comunidades_autonomas if c in address), None)
    else:
        provincia = None
        comunidad_autonoma = None

    df.at[index, 'Provincia'] = provincia
    df.at[index, 'Comunidad_Autonoma'] = comunidad_autonoma

# Guardar los cambios en un nuevo archivo CSV
df.to_csv('Data/Data_Fires_location.csv', index=False)

print("Proceso completado. Se han creado las columnas 'Provincia' y 'Comunidad_Autonoma'.")
print("El archivo modificado se ha guardado en 'Data/Data_Fires_with_locations.csv'.")


/var/folders/7s/xkdgs46x3k5636bxx89xwtqh0000gn/T/ipykernel_4872/2063513018.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/Data_Fires_location.csv')
Procesando ubicaciones: 100%|██████████| 60536/60536 [00:07<00:00, 8283.82it/s] 


Proceso completado. Se han creado las columnas 'Provincia' y 'Comunidad_Autonoma'.
El archivo modificado se ha guardado en 'Data/Data_Fires_with_locations.csv'.


In [7]:
df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,location_info,Provincia,Comunidad_Autonoma
0,43.52650,-5.73400,302.90,1.10,1.10,2008-01-01,1112,Terra,MODIS,54,6.2,282.50,9.70,D,2.0,NaN,NaN,NaN
1,37.80150,-3.61460,307.60,1.00,1.00,2008-01-01,1113,Terra,MODIS,66,6.2,278.90,11.00,D,0.0,"Camino Angosto, Mancha Real, Jaén, Andalucía, ...",Jaén,Andalucía
2,37.18340,-6.66970,300.20,1.20,1.10,2008-01-04,1322,Aqua,MODIS,33,6.2,286.70,6.00,D,0.0,"Almonte, El Condado, Huelva, Andalucía, España",Huelva,Andalucía
3,43.28080,-4.18960,300.30,1.00,1.00,2008-01-04,1323,Aqua,MODIS,34,6.2,284.40,6.50,D,0.0,"Pulero, Mazcuerras, Cantabria, 39509, España",Cantabria,Cantabria
4,37.44830,-1.77940,301.00,1.20,1.10,2008-01-05,1049,Terra,MODIS,44,6.2,279.80,7.40,D,0.0,"Pulpí, Almería, Andalucía, España",Almería,Andalucía
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60531,37.09133,-3.79891,307.26,1.21,1.09,2023-12-30,1335,Aqua,MODIS,65,6.1NRT,290.22,8.72,D,NaN,"Camino de La Malahá, Ácula, Ventas de Huelma, ...",Granada,Andalucía
60532,39.63990,-3.80005,301.17,1.11,1.05,2023-12-30,1337,Aqua,MODIS,45,6.1NRT,288.41,4.98,D,NaN,"Manzaneque, Castilla-La Mancha, España",None,Castilla-La Mancha
60533,40.40717,-1.20839,304.66,1.00,1.00,2023-12-30,1337,Aqua,MODIS,59,6.1NRT,285.20,7.60,D,NaN,"El Torrejón, Caudé, Teruel, Comunidad de Terue...",Teruel,Aragón
60534,43.52096,-5.74018,300.38,1.15,1.07,2023-12-30,1337,Aqua,MODIS,36,6.1NRT,287.94,6.64,D,NaN,"Carretera de Veriña a San Andrés, La Cruz, Gij...",Asturias,None


In [8]:
import pandas as pd
from tqdm import tqdm

# Cargar el archivo CSV
ruta_archivo = 'Data/Data_Fires_location.csv'
df = pd.read_csv(ruta_archivo)

# Extraer provincias y comunidades autónomas únicas
provincias_unicas = df['Provincia'].unique()
comunidades_autonomas_unicas = df['Comunidad_Autonoma'].unique()

# Dataframe con filas que no tienen provincia, comunidad autónoma, o ambas
df_sin_provincia_o_comunidad = df[df['Provincia'].isna() | df['Comunidad_Autonoma'].isna()]

# Contar el número de filas sin provincia, comunidad autónoma, o ambas
num_filas_sin_provincia_o_comunidad = len(df_sin_provincia_o_comunidad)

# Contar el número de filas sin provincia y sin comunidad autónoma por separado
num_filas_sin_provincia = len(df[df['Provincia'].isna()])
num_filas_sin_comunidad_autonoma = len(df[df['Comunidad_Autonoma'].isna()])

# Contar el número de provincias únicas y comunidades autónomas únicas
num_provincias_unicas = len(provincias_unicas)
num_comunidades_autonomas_unicas = len(comunidades_autonomas_unicas)

# Crear una barra de progreso para el proceso
for _ in tqdm(range(100), desc="Procesando"):
    pass

# Resultados
print("Número de Filas sin Provincia ni Comunidad Autónoma:", num_filas_sin_provincia_o_comunidad)
print("Número de Filas sin Provincia:", num_filas_sin_provincia)
print("Número de Filas sin Comunidad Autónoma:", num_filas_sin_comunidad_autonoma)
print("Número de Provincias Únicas:", num_provincias_unicas)
print("Número de Comunidades Autónomas Únicas:", num_comunidades_autonomas_unicas)

df_sin_provincia_o_comunidad

/var/folders/7s/xkdgs46x3k5636bxx89xwtqh0000gn/T/ipykernel_4872/2709161833.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo)
Procesando: 100%|██████████| 100/100 [00:00<00:00, 419850.25it/s]

Número de Filas sin Provincia ni Comunidad Autónoma: 17799
Número de Filas sin Provincia: 4779
Número de Filas sin Comunidad Autónoma: 16297
Número de Provincias Únicas: 46
Número de Comunidades Autónomas Únicas: 14


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,location_info,Provincia,Comunidad_Autonoma
0,43.52650,-5.73400,302.90,1.10,1.10,2008-01-01,1112,Terra,MODIS,54,6.2,282.50,9.70,D,2.0,NaN,NaN,NaN
5,43.53800,-5.83130,300.50,1.00,1.00,2008-01-06,1130,Terra,MODIS,38,6.2,289.90,4.20,D,0.0,"Carretera de L'Empalme a Avilés, Tabaza, Carre...",Asturias,NaN
6,38.04660,-1.68060,301.40,1.00,1.00,2008-01-06,1309,Aqua,MODIS,47,6.2,290.00,4.00,D,0.0,"Calle Hospital, Bullas, Noroeste, 30180, España",NaN,NaN
7,38.04480,-1.69210,312.80,1.00,1.00,2008-01-06,1309,Aqua,MODIS,73,6.2,289.60,11.90,D,0.0,"Vía Verde del Noroeste, Bullas, Noroeste, 3018...",NaN,NaN
9,43.22240,-3.20250,310.10,1.10,1.00,2008-01-10,1106,Terra,MODIS,69,6.2,281.50,12.90,D,0.0,"Carretera Acceso Lagarbea, La Herbosa, Artzent...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60528,38.72263,-0.72054,304.89,1.10,1.05,2023-12-25,1325,Aqua,MODIS,31,6.1NRT,290.23,6.62,D,NaN,"El Salse, Banyeres de Mariola, l'Alcoià, Alaca...",Alicante,NaN
60529,39.41438,2.80341,325.50,1.11,1.05,2023-12-27,1310,Aqua,MODIS,79,6.1NRT,289.94,26.55,D,NaN,"Camí de s'Àguila, Llucmajor, Migjorn, Illes Ba...",NaN,NaN
60532,39.63990,-3.80005,301.17,1.11,1.05,2023-12-30,1337,Aqua,MODIS,45,6.1NRT,288.41,4.98,D,NaN,"Manzaneque, Castilla-La Mancha, España",NaN,Castilla-La Mancha
60534,43.52096,-5.74018,300.38,1.15,1.07,2023-12-30,1337,Aqua,MODIS,36,6.1NRT,287.94,6.64,D,NaN,"Carretera de Veriña a San Andrés, La Cruz, Gij...",Asturias,NaN


In [9]:
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """
""" FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO """


' FINAL DE LA ESPERA, A PARTIR DE AQUÍ YA PUEDES EJECTURAR EL CODIGO SIN MIEDO '

---------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------

Filtrado de Datos de Galicia y Asignación de Estaciones Meteorológicas: ----------------------

In [73]:
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm import tqdm

In [74]:
import pandas as pd

# Ruta al archivo CSV original
fires_file_path = 'Data/Data_Fires_location.csv'

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(fires_file_path)

# Filtrar filas donde "Comunidad_Autonoma" sea igual a "Galicia"
df_galicia = df[df['Comunidad_Autonoma'] == 'Galicia']

# Guardar el DataFrame resultante en un archivo CSV
output_file_path = 'Data/data_Galicia/Data_Fires_Galicia_location.csv'
df_galicia.to_csv(output_file_path, index=False)

print("Archivo CSV de incendios en Galicia guardado con éxito.")

df_galicia

Archivo CSV de incendios en Galicia guardado con éxito.


/var/folders/7s/xkdgs46x3k5636bxx89xwtqh0000gn/T/ipykernel_4872/3865383580.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fires_file_path)


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,location_info,Provincia,Comunidad_Autonoma
55,42.31990,-6.92590,302.60,1.10,1.00,2008-01-24,1118,Terra,MODIS,53,6.2,282.20,8.70,D,0.0,"Valdemeao, Lamalonga, A Veiga, Valdeorras, Our...",Ourense,Galicia
57,42.32720,-6.93070,303.20,1.10,1.00,2008-01-24,1118,Terra,MODIS,55,6.2,281.70,8.60,D,0.0,"Prado de Borraxes, Riodolas, Carballeda de Val...",Ourense,Galicia
62,42.31280,-6.92730,302.60,2.10,1.40,2008-01-24,1259,Aqua,MODIS,53,6.2,283.80,21.30,D,0.0,"As Pedradas, Meixide, A Veiga, Valdeorras, Our...",Ourense,Galicia
78,42.14210,-7.15940,312.90,1.10,1.00,2008-01-25,1342,Aqua,MODIS,73,6.2,273.30,17.10,D,0.0,"Estrada de Fornelos a Ardexarxe, Fornelos de F...",Ourense,Galicia
79,41.95550,-7.97160,303.30,1.00,1.00,2008-01-25,1342,Aqua,MODIS,52,6.2,283.60,7.30,D,0.0,"OU-1205, Muíños, Mugueimes, A Baixa Limia, Our...",Ourense,Galicia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60247,42.20914,-7.22658,440.87,1.03,1.01,2023-10-12,2205,Terra,MODIS,100,6.1NRT,311.05,624.55,N,NaN,"A Tapada Grande, Chaguazoso, Vilariño de Conso...",Ourense,Galicia
60248,42.20757,-7.23871,384.14,1.03,1.01,2023-10-12,2205,Terra,MODIS,100,6.1NRT,292.67,177.85,N,NaN,"O Curro, Chaguazoso, Vilariño de Conso, Viana,...",Ourense,Galicia
60249,42.59256,-7.88508,307.07,1.01,1.00,2023-10-12,2205,Terra,MODIS,30,6.1NRT,285.03,9.24,N,NaN,"O Vilar, Esmeriz, Chantada, Lugo, Galicia, 275...",Lugo,Galicia
60250,42.60135,-7.88741,346.99,1.01,1.00,2023-10-12,2205,Terra,MODIS,100,6.1NRT,289.12,58.94,N,NaN,"Agro de Esmeriz, O Vilar, Esmeriz, Chantada, L...",Lugo,Galicia


Añadir la estación más cercana a cada incendio

In [75]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="prediccion_incendios")

lugares = ["Val do Dubra, A Coruña, Spain", "Boiro, A Coruña, Spain","Lousame, A Coruña, Spain","Narón, A Coruña, Spain", 
           "Castrelo de Miño, Ourense, Spain", "A Veiga, Ourense, Spain", "Calvos de Randin, Ourense, Spain", "Ríos, Ourense, Spain",
           "Baleira, Lugo, Spain", "Mondoñedo, Lugo, Spain", "Antas de Ulla, Lugo, Spain", "Chantada, Lugo, Spain", 
            "Ponteareas, Pontevedra, Spain", "Meis, Pontevedra, Spain",  "O Rosal, Pontevedra, Spain", "Lalín, Pontevedra, Spain"]

coordenadas = {}

for lugar in lugares:
    # Dividir la cadena en partes separadas por comas y obtener el primer elemento (el nombre del pueblo)
    nombre_pueblo = lugar.split(',')[0]
    
    location = geolocator.geocode(lugar)
    if location:
        # Usar solo el nombre del pueblo como clave en el diccionario
        coordenadas[nombre_pueblo] = (location.latitude, location.longitude)

# Mostrar el diccionario con los nombres de los pueblos y sus coordenadas
print(coordenadas)

{'Val do Dubra': (43.0101545, -8.6516271), 'Boiro': (42.6823024, -8.8909873), 'Lousame': (42.7733607, -8.832957073089341), 'Narón': (43.5010981, -8.1923091), 'Castrelo de Miño': (42.2965639, -8.065678393458727), 'A Veiga': (42.2507678, -7.0260508), 'Calvos de Randin': (41.92502975, -7.859098930350559), 'Ríos': (41.9745769, -7.2823706), 'Baleira': (43.05400255, -7.228073882860878), 'Mondoñedo': (43.4296687, -7.3638814), 'Antas de Ulla': (42.7674234, -7.8984065), 'Chantada': (42.6089783, -7.8218597), 'Ponteareas': (42.1741836, -8.5050427), 'Meis': (42.4996133, -8.724936507774776), 'O Rosal': (41.936991, -8.8370358), 'Lalín': (42.6513299, -8.154379)}


In [78]:
import folium

# Coordenadas centrales de Galicia para centrar el mapa
lat_galicia = 42.5751
lon_galicia = -8.1339

# Crear el mapa centrado en Galicia
mapa = folium.Map(location=[lat_galicia, lon_galicia], zoom_start=9)

# Agregar marcadores para cada ubicación en el diccionario de coordenadas
for lugar, coordenada in coordenadas.items():
    folium.Marker(coordenada, popup=lugar).add_to(mapa)

# Mostrar el mapa
mapa


In [109]:
# Cargar el dataset
file_path = 'Data/data_Galicia/Data_Fires_Galicia_location.csv'
df = pd.read_csv(file_path)


In [110]:
from geopy.distance import geodesic
from tqdm import tqdm

# Crear una función para encontrar la estación más cercana
def estacion_mas_cercana(row):
    distancias = {}
    for lugar, coordenada in coordenadas.items():
        distancias[lugar] = geodesic((row['latitude'], row['longitude']), coordenada).kilometers
    estacion_cercana = min(distancias, key=distancias.get)
    return estacion_cercana

# Aplicar la función a cada fila del DataFrame para encontrar la estación más cercana
df['Estacion'] = ''

for idx, row in tqdm(df.iterrows(), total=len(df)):
    df.at[idx, 'Estacion'] = estacion_mas_cercana(row)
    
df

100%|██████████| 8472/8472 [00:23<00:00, 354.89it/s]


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,location_info,Provincia,Comunidad_Autonoma,Estacion
0,42.31990,-6.92590,302.60,1.10,1.00,2008-01-24,1118,Terra,MODIS,53,6.2,282.20,8.70,D,0.0,"Valdemeao, Lamalonga, A Veiga, Valdeorras, Our...",Ourense,Galicia,A Veiga
1,42.32720,-6.93070,303.20,1.10,1.00,2008-01-24,1118,Terra,MODIS,55,6.2,281.70,8.60,D,0.0,"Prado de Borraxes, Riodolas, Carballeda de Val...",Ourense,Galicia,A Veiga
2,42.31280,-6.92730,302.60,2.10,1.40,2008-01-24,1259,Aqua,MODIS,53,6.2,283.80,21.30,D,0.0,"As Pedradas, Meixide, A Veiga, Valdeorras, Our...",Ourense,Galicia,A Veiga
3,42.14210,-7.15940,312.90,1.10,1.00,2008-01-25,1342,Aqua,MODIS,73,6.2,273.30,17.10,D,0.0,"Estrada de Fornelos a Ardexarxe, Fornelos de F...",Ourense,Galicia,A Veiga
4,41.95550,-7.97160,303.30,1.00,1.00,2008-01-25,1342,Aqua,MODIS,52,6.2,283.60,7.30,D,0.0,"OU-1205, Muíños, Mugueimes, A Baixa Limia, Our...",Ourense,Galicia,Calvos de Randin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8467,42.20914,-7.22658,440.87,1.03,1.01,2023-10-12,2205,Terra,MODIS,100,6.1NRT,311.05,624.55,N,NaN,"A Tapada Grande, Chaguazoso, Vilariño de Conso...",Ourense,Galicia,A Veiga
8468,42.20757,-7.23871,384.14,1.03,1.01,2023-10-12,2205,Terra,MODIS,100,6.1NRT,292.67,177.85,N,NaN,"O Curro, Chaguazoso, Vilariño de Conso, Viana,...",Ourense,Galicia,A Veiga
8469,42.59256,-7.88508,307.07,1.01,1.00,2023-10-12,2205,Terra,MODIS,30,6.1NRT,285.03,9.24,N,NaN,"O Vilar, Esmeriz, Chantada, Lugo, Galicia, 275...",Lugo,Galicia,Chantada
8470,42.60135,-7.88741,346.99,1.01,1.00,2023-10-12,2205,Terra,MODIS,100,6.1NRT,289.12,58.94,N,NaN,"Agro de Esmeriz, O Vilar, Esmeriz, Chantada, L...",Lugo,Galicia,Chantada


In [123]:
# Guardar el DataFrame resultante en un archivo CSV
output_file_path = 'Data/data_Galicia/Data_Fires_Galicia_Station.csv'
df.to_csv(output_file_path, index=False)

----------------------------------------------------------------------------------------------------------------------------------------------------------------

Código para unir todos las variables climáticas que encuentra dentro de la carpeta de estaciones metorológicas, dentro de cada provincia de forma automática

In [135]:
import pandas as pd
import os
from tqdm.auto import tqdm

def encontrar_carpeta(base_dir, nombre_carpeta):
    for root, dirs, _ in os.walk(base_dir):
        if nombre_carpeta in dirs:
            return os.path.join(root, nombre_carpeta)
    return None

base_dir = "Data/data_clima"
provincias_dir = encontrar_carpeta(base_dir, 'provincias')
if not provincias_dir:
    raise Exception(f"No se encontró la carpeta 'provincias' dentro de {base_dir}.")

output_dir = "Data/data_Galicia/estaciones_meteorologicas_completa"
os.makedirs(output_dir, exist_ok=True)

tipos_datos = ['humedad_foliar', 'humedad_relativa', 'humedad_suelo', 'precipitacion', 'temperatura_suelo', 'viento']

estaciones_procesadas = 0
provincias_procesadas = 0

for provincia in tqdm(os.listdir(provincias_dir), desc='Provincias'):
    provincia_path = os.path.join(provincias_dir, provincia)
    estaciones_path = os.path.join(provincia_path, 'estaciones_meteorologicas')
    if os.path.isdir(estaciones_path):
        provincias_procesadas += 1
        for estacion in tqdm(os.listdir(estaciones_path), desc=f'Estaciones en {provincia}', leave=False):
            estacion_path = os.path.join(estaciones_path, estacion)
            if os.path.isdir(estacion_path):
                dfs = {}
                for tipo_dato in tipos_datos:
                    tipo_dato_path = os.path.join(estacion_path, tipo_dato)
                    if os.path.isdir(tipo_dato_path):
                        archivos = [f for f in os.listdir(tipo_dato_path) if f.endswith('.csv')]
                        df_temporal = pd.DataFrame()
                        for archivo in archivos:
                            archivo_path = os.path.join(tipo_dato_path, archivo)
                            try:
                                df_actual = pd.read_csv(archivo_path)
                                df_temporal = pd.concat([df_temporal, df_actual], ignore_index=True)
                            except Exception as e:
                                print(f"Error al leer el archivo {archivo}: {e}")
                        if not df_temporal.empty:
                            clave = f"{tipo_dato}"
                            dfs[clave] = df_temporal
                if dfs:
                    df_combinado = pd.concat(dfs.values(), axis=1)
                    df_combinado = df_combinado.loc[:,~df_combinado.columns.duplicated()]
                    output_filename = f"{estacion}_combinado.csv"
                    df_combinado.to_csv(os.path.join(output_dir, output_filename), index=False)
                    estaciones_procesadas += 1
                else:
                    print(f"No se encontraron archivos para la estación {estacion} en la provincia {provincia}.")

print(f"Proceso completado. Número total de provincias procesadas: {provincias_procesadas}.")
print(f"Número total de estaciones meteorológicas procesadas: {estaciones_procesadas}.")


Provincias:   0%|          | 0/5 [00:00<?, ?it/s]

Estaciones en CO:   0%|          | 0/5 [00:00<?, ?it/s]

Estaciones en PO:   0%|          | 0/5 [00:00<?, ?it/s]

Estaciones en LU:   0%|          | 0/5 [00:00<?, ?it/s]

Estaciones en OU:   0%|          | 0/5 [00:00<?, ?it/s]

Proceso completado. Número total de provincias procesadas: 4.
Número total de estaciones meteorológicas procesadas: 16.


----------------------------------------------------------------------------------------------------------------------------------------------------------------

Ordena las filas por fecha, las columnas, añade una columna de estación y revisa que no hay filas duplicadas de cada CSV para que muestre bien la información por fechas

In [136]:
import pandas as pd
import os
from tqdm.auto import tqdm

def procesar_csv(directorio, columna_fecha):
    archivos_csv = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.csv')]
    total_archivos = len(archivos_csv)
    print(f"Total de archivos CSV encontrados: {total_archivos}")

    for archivo in tqdm(archivos_csv, desc='Procesando archivos CSV'):
        ruta_completa = os.path.join(directorio, archivo)

        try:
            df = pd.read_csv(ruta_completa)
        except Exception as e:
            print(f"Error al leer {archivo}: {e}")
            continue

        if columna_fecha in df.columns:
            try:
                # Renombrar columna de fecha y convertir a formato datetime
                df.rename(columns={columna_fecha: "Fecha"}, inplace=True)
                df['Fecha'] = pd.to_datetime(df['Fecha'])
                df.sort_values(by='Fecha', inplace=True)
                df.drop_duplicates(inplace=True)

                # Agregar columna con el nombre de la estación
                nombre_estacion = archivo.replace('_combinado.csv', '').replace('_', ' ')
                df['Estacion'] = nombre_estacion

                # Renombrar otras columnas
                nombres_columnas = {
                    "Horas humidade foliar diaria": "Humedad foliar",
                    "Humedad rel": "Humedad relativa",
                    "Humidade relativa media a\xa01.5m": "Humedad relativa",
                    "Humidade do solo": "Humedad suelo",
                    "Chuvia": "Precipitación",
                    "Temperatura do solo a -0.1m": "Temperatura del suelo",
                    "Velocidade do vento a 10m": "Viento",
                    "Velocidade do vento a 2m": "Viento"
                }
                df.rename(columns=nombres_columnas, inplace=True)
                
                # Fusión de columnas duplicadas si existen
                if "Humedad relativa.1" in df.columns:
                    df['Humedad relativa'] = df['Humedad relativa'].fillna(df['Humedad relativa.1'])
                    df.drop(columns=['Humedad relativa.1'], inplace=True)

            except Exception as e:
                print(f"Error al procesar {archivo}: {e}")
                continue

            df.to_csv(ruta_completa, index=False)
            print(f"Archivo {archivo} actualizado y guardado correctamente.")
        else:
            print(f"La columna 'Fecha' no se encuentra en el archivo {archivo}.")

directorio_csv = "Data/data_Galicia/estaciones_meteorologicas_completa"

procesar_csv(directorio_csv, "Instante lectura")


Total de archivos CSV encontrados: 16


Procesando archivos CSV:   0%|          | 0/16 [00:00<?, ?it/s]

Archivo O_Rosal_combinado.csv actualizado y guardado correctamente.
Archivo Castrelo_de_Mino_combinado.csv actualizado y guardado correctamente.
Archivo Val_do_Dubra_combinado.csv actualizado y guardado correctamente.
Archivo Rios_combinado.csv actualizado y guardado correctamente.
Archivo Mondoñedo_combinado.csv actualizado y guardado correctamente.
Archivo Naron_combinado.csv actualizado y guardado correctamente.
Archivo Antas_de_Ulla_combinado.csv actualizado y guardado correctamente.
Archivo A_veiga_combinado.csv actualizado y guardado correctamente.
Archivo Lalin_combinado.csv actualizado y guardado correctamente.
Archivo Meis_combinado.csv actualizado y guardado correctamente.
Archivo Calvos_de_randin_combinado.csv actualizado y guardado correctamente.
Archivo Baleira_combinado.csv actualizado y guardado correctamente.
Archivo Lousame_combinado.csv actualizado y guardado correctamente.
Archivo Boiro_combinado.csv actualizado y guardado correctamente.
Archivo Ponteareas_combinado

Hay archivos que a veces no hacen caso del cambio de la columna por lo que hay que cambiarlos a mano y este codigo se puede modificar si fuera necesario

In [137]:
import pandas as pd
import os
from tqdm.auto import tqdm

def renombrar_y_combinar_columnas_csv(directorio, nombres_columnas):
    archivos_csv = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.csv')]
    total_archivos = len(archivos_csv)
    print(f"Total de archivos CSV para procesar: {total_archivos}")

    for archivo in tqdm(archivos_csv, desc='Procesando archivos CSV'):
        ruta_completa = os.path.join(directorio, archivo)
        
        try:
            df = pd.read_csv(ruta_completa)
            # Renombrar columnas según el mapeo proporcionado
            columnas_a_renombrar = {col: nombres_columnas[col] for col in nombres_columnas if col in df.columns}
            if columnas_a_renombrar:
                df.rename(columns=columnas_a_renombrar, inplace=True)
            
            # Combinar "Humedad relativa" y "Humedad relativa.1" si ambas existen
            if "Humedad relativa" in df.columns and "Humedad relativa.1" in df.columns:
                df['Humedad relativa'] = df['Humedad relativa'].fillna(df['Humedad relativa.1'])
                df.drop(columns=['Humedad relativa.1'], inplace=True)

            df.to_csv(ruta_completa, index=False)
            print(f"Archivo {archivo} actualizado con nuevas columnas.")
        except Exception as e:
            print(f"Error al leer y actualizar {archivo}: {e}")

directorio_csv = "Data/data_Galicia/estaciones_meteorologicas_completa"
nombres_columnas = {
    "Humedad rel": "Humedad relativa",
    "Velocidade do vento a 10m": "Viento",
    "Humidade relativa media a\xa01.5m": "Humedad relativa"
}

renombrar_y_combinar_columnas_csv(directorio_csv, nombres_columnas)


Total de archivos CSV para procesar: 16


Procesando archivos CSV:   0%|          | 0/16 [00:00<?, ?it/s]

Archivo O_Rosal_combinado.csv actualizado con nuevas columnas.
Archivo Castrelo_de_Mino_combinado.csv actualizado con nuevas columnas.
Archivo Val_do_Dubra_combinado.csv actualizado con nuevas columnas.
Archivo Rios_combinado.csv actualizado con nuevas columnas.
Archivo Mondoñedo_combinado.csv actualizado con nuevas columnas.
Archivo Naron_combinado.csv actualizado con nuevas columnas.
Archivo Antas_de_Ulla_combinado.csv actualizado con nuevas columnas.
Archivo A_veiga_combinado.csv actualizado con nuevas columnas.
Archivo Lalin_combinado.csv actualizado con nuevas columnas.
Archivo Meis_combinado.csv actualizado con nuevas columnas.
Archivo Calvos_de_randin_combinado.csv actualizado con nuevas columnas.
Archivo Baleira_combinado.csv actualizado con nuevas columnas.
Archivo Lousame_combinado.csv actualizado con nuevas columnas.
Archivo Boiro_combinado.csv actualizado con nuevas columnas.
Archivo Ponteareas_combinado.csv actualizado con nuevas columnas.
Archivo Chantada_combinado.csv a

Concatenamos todos los CSV de las estaciones meteorologicas

In [138]:
import pandas as pd
import os
from tqdm.auto import tqdm

def unir_csvs_en_uno(directorio, archivo_salida, columna_fecha):
    archivos_csv = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.csv')]
    total_archivos = len(archivos_csv)
    print(f"Total de archivos CSV para unir: {total_archivos}")

    lista_dfs = []
    for archivo in tqdm(archivos_csv, desc='Uniendo archivos CSV'):
        ruta_completa = os.path.join(directorio, archivo)
        try:
            df = pd.read_csv(ruta_completa)
            lista_dfs.append(df)
        except Exception as e:
            print(f"Error al leer {archivo}: {e}")

    df_unido = pd.concat(lista_dfs, ignore_index=True)
    # Convertir la columna de fecha y ordenar
    df_unido[columna_fecha] = pd.to_datetime(df_unido[columna_fecha])
    df_unido.sort_values(by=columna_fecha, inplace=True)
    
    df_unido.to_csv(archivo_salida, index=False)
    print(f"Archivo {archivo_salida} creado y ordenado por fecha con éxito.")

directorio_csv = "Data/data_Galicia/estaciones_meteorologicas_completa"
archivo_salida = "Data/data_Galicia/clima_estaciones.csv"
columna_fecha = "Fecha"  # Asegúrate de que este es el nombre correcto de la columna de fecha

unir_csvs_en_uno(directorio_csv, archivo_salida, columna_fecha)


Total de archivos CSV para unir: 16


Uniendo archivos CSV:   0%|          | 0/16 [00:00<?, ?it/s]

Archivo Data/data_Galicia/clima_estaciones.csv creado y ordenado por fecha con éxito.


In [139]:
import pandas as pd

# Cargar el archivo combinado
archivo_salida = "Data/data_Galicia/clima_estaciones.csv"
df_unido = pd.read_csv(archivo_salida)

# Estadísticas generales del DataFrame combinado
estadisticas_generales = df_unido.describe()

# Mostrar estadísticas generales como DataFrame
estadisticas_generales_df = pd.DataFrame(estadisticas_generales)
estadisticas_generales_df

,Humedad foliar,Humedad relativa,Humedad suelo,Precipitación,Temperatura del suelo,Viento
count,93504.000000,93467.000000,92895.000000,93493.000000,93464.000000,93504.000000
mean,-754.357794,-543.599405,-1447.272526,-210.106051,-611.437097,-687.209863
std,2666.664812,2430.771129,3530.456778,1448.575568,2423.102461,2540.574696
min,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
25%,0.700000,71.000000,0.100000,0.000000,9.000000,3.000000
50%,8.400000,82.000000,0.200000,0.000000,13.100000,4.700000
75%,14.900000,90.000000,0.300000,3.000000,17.900000,7.200000
max,120.000000,100.000000,200.000000,3060.000000,31.100000,78.600000


In [140]:
df_unido

,Fecha,Humedad foliar,Humedad relativa,Humedad suelo,Precipitación,Temperatura del suelo,Viento,Estacion
0,2008-01-01,109.0,98.0,0.4,117.4,11.9,4.4,O Rosal
1,2008-01-01,110.0,99.0,0.2,91.6,8.1,1.2,Castrelo de Mino
2,2008-01-01,107.0,85.0,0.4,14.6,8.1,21.6,Boiro
3,2008-01-01,86.0,90.0,-9999.0,18.8,10.1,9.3,Lousame
4,2008-01-01,80.0,91.0,0.5,0.3,-9999.0,25.1,Baleira
...,...,...,...,...,...,...,...,...
93499,2023-12-31,16.8,90.0,0.3,2.4,9.9,13.5,Val do Dubra
93500,2023-12-31,16.7,86.0,0.2,9.6,11.1,2.0,Castrelo de Mino
93501,2023-12-31,11.4,90.0,0.5,15.0,11.8,7.9,O Rosal
93502,2023-12-31,16.7,83.0,0.2,3.6,14.1,1.4,Ponteareas


Combinar tanto el dataset de Incendios localizados en Galicia con el de estaciónes meteorológicas de Galicia, para crear un único conjunto de datos

In [141]:
import pandas as pd
import unicodedata
from tqdm.auto import tqdm

def normalizar_texto(texto):
    """Normaliza cadenas de texto eliminando tildes y caracteres especiales."""
    return unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('ascii').lower()

def procesar_archivos(archivo_incendios, archivo_clima, archivo_salida):
    # Cargar los datos
    df_incendios = pd.read_csv(archivo_incendios)
    df_clima = pd.read_csv(archivo_clima)

    # Convertir las fechas a formato de fecha
    df_incendios['acq_date'] = pd.to_datetime(df_incendios['acq_date'])
    df_clima['Fecha'] = pd.to_datetime(df_clima['Fecha'])

    # Normalizar nombres de estaciones
    df_incendios['Estacion'] = df_incendios['Estacion'].apply(normalizar_texto)
    df_clima['Estacion'] = df_clima['Estacion'].apply(normalizar_texto)

    # Fusionar los datasets
    print("Combinando datasets...")
    df_combinado = pd.merge(df_incendios, df_clima, left_on=['acq_date', 'Estacion'], right_on=['Fecha', 'Estacion'], how='left')

    # Eliminar columnas redundantes y renombrar según sea necesario
    df_combinado.drop(columns=['Fecha_y'], inplace=True, errors='ignore')
    df_combinado.rename(columns={'Fecha_x': 'Fecha'}, inplace=True)

    # Guardar el archivo combinado
    df_combinado.to_csv(archivo_salida, index=False)
    print(f"Archivo combinado guardado en: {archivo_salida}")

# Rutas de los archivos
archivo_incendios = 'Data/data_Galicia/Data_Fires_Galicia_Station.csv'
archivo_clima = 'Data/data_Galicia/clima_estaciones.csv'
archivo_salida = 'Data/data_Galicia/galicia_inc.csv'

# Ejecutar la función
procesar_archivos(archivo_incendios, archivo_clima, archivo_salida)


Combinando datasets...
Archivo combinado guardado en: Data/data_Galicia/galicia_inc.csv


Busca posibles estaciones meteorologicas de las cuales exista algún error 

In [142]:
import pandas as pd

# Cargar el archivo
archivo = 'Data/data_Galicia/galicia_inc.csv'  
df = pd.read_csv(archivo)

# Convertir la columna 'Fecha' a datetime y manejar valores NaN
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

# Encontrar estaciones únicas donde la fecha es NaT (Not a Time)
estaciones_con_fecha_vacia = df[df['Fecha'].isna()]['Estacion'].unique()

# Crear un DataFrame para mostrar las estaciones
df_estaciones_con_fecha_vacia = pd.DataFrame(estaciones_con_fecha_vacia, columns=['Estaciones con Fecha Vacía'])
print(df_estaciones_con_fecha_vacia)


Empty DataFrame
Columns: [Estaciones con Fecha Vacía]
Index: []
